<a href="https://colab.research.google.com/github/jerichooconnell/phys555-A2/blob/master/Copy_of_PHYS_555_DeepLearningAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Deep Learning

Please complete the two sections. Submit a copy of the notebook when run entirely, either by uploading the notebook or a restricted shared link


## A - Concepts (40%)

Answer each question as concisely as you can.

1. **For a multi-class classification neural network, what kind of activation would you use in the output layer and which loss function would you select?**

Softmax and cross entropy

2. **Give two possible methods to avoid overfitting a deep learning model. Explain very briefly how each help mitigating overfitting.**

Regularization
Dataset variability

3. **What is the main issue the attention mechanism help with if you want to use to translate sequences**

% your answer

4. You have a dataset made of 1M audio recordings. Each recording has exactly the same number of samples, and consists of one instrument.
Your task would be to obtain the type of instruments used in all the recordings.

  You are in luck, you also have the type of instrument for 10,000 of the recordings, and you know there is no other type of instrument in the other recordings. 

  Briefly explain how to formulate the problem from a machine learning perspective, i.e.:
    - 5.1. **How would you split your data set?**
    - 5.2. **Is a neural network a good choice for the problem? What type of NN?**
    - 5.3. **If you are using a NN, indicate the what is the last layer of your NN, your activation function, and which loss you would be using to train the NN.**

% your answer

## 2. Applications (60%)

#### **Noise2Noise**

In this part of the assignment, we will implement the training of "Noise2Noise: Learning Image Restoration without Clean Data". 
Most of the network is implemented. You will have to answer a few questions, and implement a few things only.


![](https://research.nvidia.com/sites/default/files/publications/n2n-representative_0.png)


The one line summary of the method is the following: we can learn to remove noise from images by training a UNet between two noisy versions of the same image.


If we had a set of noisy but also ground-truth, noiseless images of the same objects, we can imagine to learn a neural network to perform a mapping between the noisy and the clean variant. But what if we only have noisy images? This is the point of Noise2Noise.

When one has set of noisy images where the only difference is an additive and independent noise realisation, then the Noise2Noise network can produce images with much reduced noise level. One could use those _denoised_ images to detect very faint features.

See the [original paper](https://arxiv.org/abs/1803.04189) if you are interested in more details. Above is a figure from the paper showing a MRI denoised image with Noise2Noise (2nd column) which can get very similar signal to noise level as if the dataset was trained with noiseless images as targets.


Given one image $\mathbf{x}$ with two additive and unbiased noise realisations $ϵ$ ab $\delta$, i.e.:
$\mathbf{y}=\mathbf{x}+\epsilon$ and $\tilde{\mathbf{y}} = \mathbf{x} + \delta$. We can formulate our denoising task as a _regression_ problem,finding a network $f_\theta$ such that $\mathbf{\tilde{y}}$
$$
\hat{\theta} = \arg\min_\theta \mathbb{E}\Vert \mathbf{\tilde{y}} - \mathbf{y} \Vert^2
$$

1. **Show that the solution is unbiased when using $L_2$ for a loss function, for two additive and _independent_ noises, in other word**:

$$
\arg\min_\theta \mathbb{E}\left[ \Vert f_\theta(\mathbf{x}+\epsilon) - (\mathbf{x}+\delta)\Vert^2 \right] = \arg\min_\theta \mathbb{E}\left[ \Vert f_\theta(\mathbf{x}+\epsilon) - \mathbf{x}\Vert^2 \right]
$$


% your answer





Noise2Noise is a UNet, a fully convolutional (no dense layers) neural network. We will code it below and you will have to comment and modify.


First, let's do the boiler plates

In [ ]:
%matplotlib inline
%load_ext tensorboard

import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms

# for image i/o and metrics
from PIL import Image
import imageio
from skimage.metrics import peak_signal_noise_ratio

In [ ]:
# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

## Data loading and preprocessing

In order to train and validate our model, we will use natural images from the [VDSR dataset](https://cv.snu.ac.kr/research/VDSR). 

The images are noiseless but we will add artificial noise.
First let's fetch the data.

In [ ]:
!wget --no-clobber https://cv.snu.ac.kr/research/VDSR/train_data.zip && mkdir -p train && unzip -qq train_data.zip -d train
!ls -1 train | wc 
# run twice the cell if you get an error 

In [ ]:
!wget --no-clobber https://cv.snu.ac.kr/research/VDSR/test_data.zip  && mkdir -p valid && unzip -qq test_data.zip -d valid
!ls -1 valid | wc 
# rerun if you get an error 

In [ ]:
train_dir = 'train'
valid_dir = 'valid'

We first create a custom `Dataset`  class for loading all the images from disk and apply noise functions. The data is composed of various formats (bmp, jpg,...) and various sizes. We are cropping the images.

In [ ]:
# default noise model: does not add noise.
noiseless = lambda x: (x, x)

class NoisyImages(Dataset):

    def __init__(self, img_dir, add_noise=noiseless, crop_size=256):

        suffixes = (".jpeg", ".jpg", ".png", ".bmp")
        self.image_paths = [p for p in Path(img_dir).glob("**/*") if p.suffix.lower() in suffixes]
        
        self.add_noise = add_noise
        
        # the ToTensor converts a numpy array (H,W,C) in the range [0,255]
        # to a torch tensor(C,H,W) in the range [0,1]
        self.transforms = transforms.Compose([
            transforms.RandomCrop(crop_size, pad_if_needed=True, padding_mode='reflect'),
            transforms.ToTensor()
        ])
        
    # number of samples in the DataSet
    def __len__(self):
        return len(self.image_paths)

    # load image given an index of the sample
    def __getitem__(self, idx):
  
        img = imageio.imread(self.image_paths[idx])

        # here we assume a 3-channel (RGB) image.
        # single-channel images will fill other channels with a copy
        if img.ndim == 2:
            img = np.stack([img, img, img], axis=2)
        elif img.shape[2] == 1:
            img = np.concatenate([img, img, img], axis=2)

        # convert to PIL image array and transform it
        img = Image.fromarray(img)
        img = self.transforms(img)

        # offset the output transformed tensor from [0,1] to [-0.5,0.5] 
        img = img - 0.5
        
        # apply the noise function which returns a source and target image
        return self.add_noise(img)

Let us create training dataset and show some of the images. First some usual display functions to help visualizing our samples.

In [ ]:
# converts torch tensor between [-0.5,0.5] into a uint8 tensor
def clip_to_uint8(x):
    return torch.clamp((x + 0.5) * 255.0 + 0.5, 0, 255).type(torch.uint8)

# plot a sample of two noisy images
def show_random_sample(dataset):
    idx = np.random.randint(0, len(dataset))
    source, target = dataset[idx]

    # convert (C,H,W) to (H,W,C) and to uint8
    source = np.transpose(source, (1, 2, 0)) 
    target = np.transpose(target, (1, 2, 0))

    source = clip_to_uint8(source)
    target = clip_to_uint8(target)
    
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(source)
    axarr[1].imshow(target)
    _ = [ax.axis('off') for ax in axarr]
    print(f'Image size is {source.shape}')
    plt.show()

In [ ]:
# lets check some samples images out
train_data = NoisyImages(train_dir)
show_random_sample(train_data)

We need to produce noise. First case is Gaussian noise. We randomize the noise standard deviation  $\sigma \in [0, 50]$ separately for each training example. We need the noise function to add noise twice for source and target image separately.

In [ ]:
class AddGaussianNoise:
    def __init__(self, sigma_range):
        self.minval, self.maxval = sigma_range
        self.minval = self.minval / 255
        self.maxval = self.maxval / 255

    def __call__(self, x):
        sigma = (self.maxval - self.minval) * torch.rand(1) + self.minval
        return x + torch.randn(x.size()) * sigma

In [ ]:
add_noise_train = AddGaussianNoise((0, 200))

# add Gaussian noise with different standard deviation.
train_noise = lambda x: (add_noise_train(x), add_noise_train(x))

train_data = NoisyImages(train_dir, add_noise=train_noise)
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)

# same for validation set
add_noise_valid = AddGaussianNoise((100, 100))

valid_noise = lambda x: (add_noise_valid(x), x)

valid_data = NoisyImages(valid_dir, add_noise=valid_noise)
valid_loader = DataLoader(valid_data, batch_size=4, shuffle=True)

In [ ]:
# replay the cell several times
show_random_sample(train_data)
show_random_sample(valid_data)

2. **Why are the validation set target images noiseless?**

% your answer

## Neural Network Architecture 

We are going to build almost the same UNet as the paper suggests.



In [ ]:
class Noise2Noise(nn.Module):

    def __init__(self, in_channels=3, out_channels=3):

        super(Noise2Noise, self).__init__()

        # Layers: enc_conv0, enc_conv1, pool1
        self._block1 = nn.Sequential(
            nn.Conv2d(in_channels, 48, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(2))

        # Layers: enc_conv(i), pool(i); i=2..5
        self._block2 = nn.Sequential(
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(2))

        # Layers: enc_conv6, upsample5
        self._block3 = nn.Sequential(
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.ConvTranspose2d(48, 48, kernel_size=3, stride=2, padding=1, output_padding=1))

        # Layers: dec_conv5a, dec_conv5b, upsample4
        self._block4 = nn.Sequential(
            nn.Conv2d(96, 96, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(96, 96, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))

        # Layers: dec_deconv(i)a, dec_deconv(i)b, upsample(i-1); i=4..2
        self._block5 = nn.Sequential(
            nn.Conv2d(144, 96, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(96, 96, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.ConvTranspose2d(96, 96, kernel_size=3, stride=2, padding=1, output_padding=1))

        # Layers: dec_conv1a, dec_conv1b, dec_conv1c,
        self._block6 = nn.Sequential(
            nn.Conv2d(96 + in_channels, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(32, out_channels, kernel_size=3, padding=1))
        
        # Initialize weights
        self._init_weights()


    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
                m.bias.data.zero_()

    def forward(self, x):

        # Encoder
        pool1 = self._block1(x)
        pool2 = self._block2(pool1)
        pool3 = self._block2(pool2)
        pool4 = self._block2(pool3)
        pool5 = self._block2(pool4)

        # Decoder
        upsample5 = self._block3(pool5)
        concat5 = torch.cat((upsample5, pool4), dim=1)

        upsample4 = self._block4(concat5)
        concat4 = torch.cat((upsample4, pool3), dim=1)
        
        upsample3 = self._block5(concat4)
        concat3 = torch.cat((upsample3, pool2), dim=1)
        
        upsample2 = self._block5(concat3)
        concat2 = torch.cat((upsample2, pool1), dim=1)
        
        upsample1 = self._block5(concat2)
        concat1 = torch.cat((upsample1, x), dim=1)

        return self._block6(concat1)

3. Architecture understanding:

> a. **What is the activation of the output layer? Why?**

> b. **Is there any skip connection in the network? If yes, where?**

> c. **Why do we have padding=1 in the encoder layers?**

> d. **Which layer is reducing the dimension of the ?**


% your answers

Check the network is working with a single forward pass on a random image

In [ ]:
model = Noise2Noise(in_channels=3, out_channels=3)

idx = np.random.randint(0, len(train_data))
img = train_data[idx][0]
output = model(img.unsqueeze(0))

## Training

Now our training and validation routines

In [ ]:
# apply training for one epoch
def train_model(model, loader, optimizer, criterion,
                epoch, log_interval=100, tb_logger=None):

    # set the model to train mode
    model.train()

    # iterate over the batches of this epoch
    for batch_idx, (x, y) in enumerate(loader):

        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        # apply model, calculate loss and run backwards pass
        prediction = model(x)
        loss = criterion(prediction, y)
        loss.backward()
        optimizer.step()
        
        # log to notebook
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(x),
                  len(loader.dataset),
                  100. * batch_idx / len(loader), loss.item()))

            # log loss and images to tensorboard
            if tb_logger is not None:
                step = epoch * len(loader) + batch_idx
                tb_logger.add_scalar(tag='train_loss', scalar_value=loss.item(), global_step=step)
                
                x, y, prediction = clip_to_uint8(x), clip_to_uint8(y), clip_to_uint8(prediction)
                tb_logger.add_images(tag='input', img_tensor=x.to('cpu'), global_step=step)
                tb_logger.add_images(tag='target', img_tensor=y.to('cpu'), global_step=step)
                tb_logger.add_images(tag='prediction', img_tensor=prediction.to('cpu').detach(), global_step=step)

Validation: we need a metric to evaluate our model besides the loss. One possible quick one is the [Peak signal-to-noise ratio](https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio) will be used from the `scikit-image` module.

In [ ]:
class psnr:
    def __call__(self, image_true, image_pred):
        image_true = clip_to_uint8(image_true).detach().cpu().numpy()
        image_pred = clip_to_uint8(image_pred).detach().cpu().numpy()
        return peak_signal_noise_ratio(image_true, image_pred)


In [ ]:
def valid_model(model, loader, criterion, metric, step=None, tb_logger=None):
    
    # set model to evaluation mode
    model.eval()

    val_loss = 0
    val_metric = 0
    
    # we are not computing gradients during validation
    with torch.no_grad():

        for x, y in loader:
            x, y = x.to(device), y.to(device)
            prediction = model(x)
            loss = criterion(prediction, y)

            val_score = metric(y, prediction)
            
            val_loss += loss
            val_metric += val_score
    
    # normalize loss and metric
    val_loss /= len(loader)
    val_metric /= len(loader)
    
    if tb_logger is not None:
        assert step is not None, "Need to know the current step to log validation results"
        tb_logger.add_scalar(tag='val_loss', scalar_value=val_loss, global_step=step)
        tb_logger.add_scalar(tag='val_metric', scalar_value=val_metric, global_step=step)
        # we always log the last validation images
        x, y, prediction = clip_to_uint8(x), clip_to_uint8(y), clip_to_uint8(prediction)
        tb_logger.add_images(tag='val_input', img_tensor=x.to('cpu'), global_step=step)
        tb_logger.add_images(tag='val_target', img_tensor=y.to('cpu'), global_step=step)
        tb_logger.add_images(tag='val_prediction', img_tensor=prediction.to('cpu'), global_step=step)
        
    print('\nValidate: Average loss: {:.4f}, Average Metric: {:.4f}\n'.format(val_loss, val_metric))

In [ ]:
# Start tensorboard summary. 
# Remember to hit reload in the tensorboad, and check the images
# in the settings, you can also set the autoreload option

logger = SummaryWriter('runs/noise2noise')
%tensorboard --logdir runs

Finally we have the necessary components to train and validate our Noise2Noise on the VDSR images.


In [ ]:
model = Noise2Noise(in_channels=3, out_channels=3)
model = model.to(device)

# same parameters as the paper
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.99))

criterion = nn.MSELoss()
metric = psnr()

# start with low number of epochs to check, then increase
n_epochs = 5

for epoch in range(n_epochs):
    train_model(model, train_loader, optimizer, criterion, epoch, log_interval=25, tb_logger=logger)
    step = epoch * len(train_loader.dataset)
    valid_model(model, valid_loader, criterion, metric, step=step, tb_logger=logger)

---

You can play with the network, batch size, learning rate, more epochs, etc...
Answer the questions below.

4. **Add batch normalization to layers in the Noise2Noise model, and retrain. Compare the behaviour of the loss function.**

% your answer


5. **Switch the target to be a noiseless image, retrain, and report the PSNR and final loss behaviour with the model trained on noisy images.** 

% your answer



6. [Optional] **Train Noise2Noise with a different additive noise model, e.g. Poisson noise with varying $\lambda$ (you can use `scikit-image`), salt-and-pepper noise, or adding different noise models between source and targets. Show your metrics and a sample of denoised image or in tensorboard** 

% your answer
